In [1]:
# load NebulaGraph Jupyter extension to enable %ngql magic
%reload_ext ngql

In [2]:

# connect to NebulaGraph service
%ngql --address graphd --port 9669 --user root --password nebula

%ngql CREATE SPACE IF NOT EXISTS llamaindex_nebula_property_graph(vid_type=FIXED_STRING(256));

[OK] Connection Pool Created


""


In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
import os

# Access the environment variable
ollama_base_url = os.getenv('OLLAMA_BASE_URL', 'http://default-url:11434')
print(f"Ollama base URL: {ollama_base_url}")
ollama_llm=Ollama(model="llama3.1", json_mode=True, request_timeout=3600, base_url=ollama_base_url)
documents = SimpleDirectoryReader("/workspace/data/paul_graham/").load_data()

Ollama base URL: http://host.docker.internal:11434


In [4]:
# use the graph space, which is similar to "use database" in MySQL
# The space was created in async way, so we need to wait for a while before using it, retry it if failed
%ngql USE llamaindex_nebula_property_graph;

""


In [5]:
from typing import Literal
from llama_index.llms.ollama import Ollama
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

# best practice to use upper-case
entities = Literal["PERSON", "PLACE", "ORGANIZATION"]
relations = Literal["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"]

# define which entities can have which relations
validation_schema = {
    "PERSON": ["HAS", "PART_OF", "WORKED_ON", "WORKED_WITH", "WORKED_AT"],
    "PLACE": ["HAS", "PART_OF", "WORKED_AT"],
    "ORGANIZATION": ["HAS", "PART_OF", "WORKED_WITH"],
}
validation_schema = [
    ("ORGANIZATION", "HAS", "PERSON"),
    ("PERSON", "WORKED_AT", "ORGANIZATION"),
    ("PERSON", "WORKED_WITH", "PERSON"),
    ("PERSON", "WORKED_ON", "ORGANIZATION"),
    ("PERSON", "PART_OF", "ORGANIZATION"),
    ("ORGANIZATION", "PART_OF", "ORGANIZATION"),
    ("PERSON", "WORKED_AT", "PLACE"),
]

kg_extractor = SchemaLLMPathExtractor(
    llm=ollama_llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    # if false, allows for values outside of the schema
    # useful for using the schema as a suggestion
    strict=True,
)

In [6]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore
from llama_index.core.vector_stores.simple import SimpleVectorStore

graph_store = NebulaPropertyGraphStore(
    space="llamaindex_nebula_property_graph", overwrite=True,
    url="nebula://graphd:9669", username="root", password="nebula",
    
)
vec_store = SimpleVectorStore()

In [9]:

from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[kg_extractor],
    embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    property_graph_store=graph_store,
    vector_store=vec_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]
Generating embeddings: 0it [00:00, ?it/s]


In [13]:
%ngql --address graphd --port 9669 --user root --password nebula

[OK] Connection Pool Created


,Name
0,llamaindex_nebula_property_graph
1,llamaindex_nebula_property_graph2


In [21]:
%ngql SHOW SPACES;

,Name
0,llamaindex_nebula_property_graph
1,llamaindex_nebula_property_graph2


In [31]:
%ngql USE llamaindex_nebula_property_graph;
%ngql SHOW STATS;

,Type,Name,Count
0,Tag,Chunk__,22
1,Tag,Entity__,0
2,Tag,Node__,22
3,Tag,Props__,22
4,Edge,Relation__,0
5,Edge,__meta__node_label__,0
6,Edge,__meta__rel_label__,0
7,Space,vertices,22
8,Space,edges,0


In [39]:
%ngql FETCH PROP ON Tag "Node__";

[ERROR]:
 Query Failed:
 SyntaxError: syntax error near `Tag'
 Query:
 FETCH PROP ON Tag "Node__";



""


In [24]:
%ngql MATCH p=()-[]->() RETURN p LIMIT 20;

,p


In [40]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0

In [42]:
graph_store.get_triplets()

[]

In [12]:
from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever,
)


llm_synonym = LLMSynonymRetriever(
    index.property_graph_store,
    llm=Ollama(model="llama3.1", request_timeout=3600),
    include_text=True,
)
vector_context = VectorContextRetriever(
    index.property_graph_store,
    embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    include_text=False,
)

In [13]:
retriever = index.as_retriever(
    sub_retrievers=[
        llm_synonym,
        vector_context,
    ]
)

In [14]:
nodes = retriever.retrieve("What happened?")

for node in nodes:
    print(node.text)

In [43]:
from llama_index.core.indices.property_graph import (
    LLMSynonymRetriever,
    VectorContextRetriever,
)
import os

ollama_base_url = os.getenv('OLLAMA_BASE_URL', 'http://default-url:11434')
llm = Ollama(model="llama3.1", json_mode=True, request_timeout=3600, base_url=ollama_base_url)
llm_synonym = LLMSynonymRetriever(
    index.property_graph_store,
    llm=llm,
    include_text=False,
)
vector_context = VectorContextRetriever(
    index.property_graph_store,
    embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    include_text=False,
)
query_engine = index.as_query_engine(
    sub_retrievers=[
        llm_synonym,
        vector_context
    ],
    llm=llm,
)

response = query_engine.query("What happened at Interleaf?")

print(str(response))

Empty Response
